In [2]:
# demo and evaluate
import os
import numpy as np
import time
import datetime
from tqdm import tqdm

import cv2
import models
import torch

from torch.utils.data import DataLoader
from torchvision import datasets
import torchvision.transforms as transforms
from torch.autograd import Variable

from datasets import *
from utils import *
from paths import *

from sklearn.metrics import accuracy_score,f1_score,roc_auc_score,recall_score,precision_score

cuda = True if torch.cuda.is_available() else False
num_gpu = list(range(torch.cuda.device_count()))
generator = models.__dict__['resskspp']()

if cuda:
    generator = generator.cuda()

generator.load_state_dict(torch.load(pretrained_res_best, map_location=torch.device('cpu')))

dut_CE_dataloader = DataLoader( DefocusDataset(root=DUTDefocus,mode="test"),
    batch_size=1, shuffle=False,num_workers=1)

# Tensor type
Tensor = torch.cuda.FloatTensor if cuda else torch.FloatTensor

def val():

    dictx = {
        "dut":dut_CE_dataloader,
    }

    generator.eval()
    
    with torch.no_grad():
        for data_name, dataloaderx in dictx.items():
            mae,preds,gts = [],[],[]
            for i, batch in tqdm(iter(enumerate(dataloaderx))):
                # Model inputs
                real_A = Variable(batch["A"].type(Tensor))
                rgb_A = batch["Ao"][0]
                mask = batch["M"]
                name = batch["name"][0]

                gs,ds = generator(real_A)
                generated_depth = resize_to_match(ds[0], mask)
                pred_depth = np.clip(to_numpy(generated_depth[0,0]),0,1.0)
                depth = np.uint8(pred_depth*255)
                
                generated_mask = torch.sigmoid(gs[0])
                generated_mask = resize_to_match(generated_mask, mask)
              
                pred_float = to_numpy(generated_mask[0, 0])
                gt_float = to_numpy(mask[0])

                #MAE
                mae.append(np.sum(cv2.absdiff(gt_float.astype(float),pred_float.astype(float)))/(pred_float.shape[1]*pred_float.shape[0]))

                pred = np.uint8(pred_float * 255)
                gt = np.uint8(gt_float*255)

                pred_float_ = np.where(pred>min(1.5*np.mean(pred),255),np.ones_like(pred_float),np.zeros_like(pred_float))
                gt_float_ = np.where(gt>min(1.5*np.mean(gt),255),np.ones_like(pred_float),np.zeros_like(pred_float))

                preds.extend(pred_float_.ravel())
                gts.extend(gt_float_.ravel())

            RECALL = recall_score(gts,preds)
            PERC = precision_score(gts,preds)

            fmeasure = (1+0.3)*PERC*RECALL/(0.3*PERC+RECALL)
            MAE = np.mean(mae)
            print("MAE: %s, fmeasure: %s " % (MAE,fmeasure))
            print()
    return MAE

val()




Dataset:500


500it [28:28,  3.42s/it]


MAE: 0.07265675144741514, fmeasure: 0.8909035691654591 



0.07265675144741514